In [5]:
%%writefile tablereader_js.py
from pymongo import MongoClient
import pprint 
import pandas as pd 
import copy
from bs4 import BeautifulSoup
import requests
import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException

from time import sleep
import warnings

warnings.filterwarnings('ignore')

def page_to_mongo(site_url, collection_name):
    # send GET request using selenium (sites in javascript) and check status
    option = webdriver.ChromeOptions()
    option.add_argument(' - incognito')
    option.add_argument('--headless')

    browser = webdriver.Chrome(executable_path='/usr/local/bin/chromedriver', chrome_options=option)

    req = requests.get(site_url)
    stat_code = req.status_code
    tstamp = datetime.datetime.now().strftime('%m-%d-%Y %H:%M:%S')
    print('_______________')
    print('_______________')
    print(site_url)
    print('Request Status Code: {}'.format(stat_code))
    if req.status_code == 200:
        # add page page html to mongo
        collection_name.insert_one({'lxml': req.content})
        # print result of load
        with open('data/load_results.txt', 'a') as f:
            f.writelines('{}, {}, {}\n'.format(site_url, stat_code, tstamp))
        f.close()
        
    else: 
        print('failed to get {}'.format(site_url))
        # print result of load
        with open('data/logs/load_results.txt', 'a') as f:
            f.writelines('{}, {}, {}\n'.format(site_url, stat_code, tstamp))
        f.close()
    
    return None

        
        
        


######################

        
if __name__ == '__main__':
    # initialize mongo driver to add items as we iterate through them
    client = MongoClient('mongodb://localhost:27017/')
    db = client.bills
    pages = db.pages

        
    # the 101st Congress (1989 - 1990) starts on pg 1011 for pageSize=250
    site_url_root = 'https://www.congress.gov/search?q={%22source%22:%22legislation%22}&pageSize=250'
    for i in range(1, 1011):
        site_url = site_url_root + '&page={}'.format(i)
        sleep(5)
        rows = page_to_mongo(site_url, pages)
#         add_to_mongo(rows)


    print('Count of pages loaded: {}'.format(pages.find().count()))

Overwriting tablereader_js.py


In [1]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
db = client.bills
pages = db.pages

In [2]:
import datetime

In [4]:
datetime.datetime.now().strftime('%m-%d-%Y %H:%M:%S')

'12-04-2018 09:06:17'

In [22]:
# iterate through pages in mongo to extract relevant data
# for p in pages.find():
#     print(len(p['lxml']))

In [18]:
import pprint
from bs4 import BeautifulSoup

p = pages.find_one()['lxml']
soup = BeautifulSoup(p, 'lxml')
# print(soup.prettify())

# table of bills are in ol class
div = soup.find('div', {'class':'search-column-main'})
table = div.find('ol')
# print(table.prettify())

# iterate though each li class expanded to get rows
rows = table.find_all('li', {'class':'expanded'})
print(rows[0].prettify())

<li class="expanded">
 <div>
  <span class="visualIndicator">
   BILL
  </span>
 </div>
 1.
 <span class="result-heading">
  <a href="https://www.congress.gov/bill/115th-congress/house-bill/7211?r=1">
   H.R.7211
  </a>
  — 115th Congress (2017-2018)
 </span>
 <span class="result-title">
  To amend the Mineral Leasing Act to authorize the Secretary of the Interior to regulate hydraulic fracturing operations on Federal lands, and for other purposes.
 </span>
 <span class="result-item">
  <strong>
   Sponsor:
  </strong>
  <a href="/member/darren-soto/S001200" target="_blank">
   Rep. Soto, Darren [D-FL-9]
  </a>
  (Introduced 11/30/2018)
  <strong>
   Cosponsors:
  </strong>
  (
  <a href="https://www.congress.gov/bill/115th-congress/house-bill/7211/cosponsors?r=1&amp;overview=closed#tabs">
   0
  </a>
  )
 </span>
 <span class="result-item">
  <strong>
   Committees:
  </strong>
  House - Natural Resources, Energy and Commerce
 </span>
 <span class="result-item">
  <strong>
   Latest A

In [70]:
columns = row.find_all('span')
t = columns[3].text.strip().split()
'(Introduced' in t
# t = t[t.index('(Introduced') + 1][:-1]
# t

True

In [81]:
# store each row as key-value pair in a dictionary
empty_row = {'leg_id': None, 
            'leg_type': None,
            'leg_url': None,
            'intro_date': None,
            'congress_id': None,
            'desc': None,
            'sponsor': None, 
            'sponsor_party': None, 
            'sponsor_state': None,
            'sponsor_district': None,  #senators don't have districts
            'num_of_cosponsors': None,
            'cosponsors_url': None,
            'cosponsors': None,  #requires navigation to another url and extracting names from table
            'committee': None, 
            'bill_status': None,
            'body': None   #requires navigation to another url
            }


In [82]:
import copy
len(rows)

250

In [85]:
all_rows = []

# iterate through each of the 'rows' to fill in the 'columns'
for row in rows:
    new_row = copy.copy(empty_row)

    # parse items within a tag
    columns = row.find_all('a')
    if columns[0].text.strip() != '':
        new_row['leg_id'] = columns[0].text.strip()
    if columns[0]['href'].strip() != '':
        new_row['leg_url'] = columns[0]['href'].strip()
    if columns[1].text.strip() != '':
        new_row['sponsor'] = columns[1].text.strip()
    if columns[2].text.strip() != '':
        new_row['num_of_cosponsors'] = columns[2].text.strip()    

    # parse items within span tag
    columns = row.find_all('span')
    if columns[1].text.strip().split()[2] != '':
        new_row['congress_id'] = columns[1].text.strip().split()[2]
    if columns[2].text != '':
        new_row['desc'] = columns[2].text
    if columns[4].text.strip()[12:] != '':
        new_row['committee'] = columns[4].text.strip()[12:]
    # date was a little tricky
    dt = columns[3].text.strip().split()
    if '(Introduced' in dt:
        new_row['intro_date'] = dt[dt.index('(Introduced') + 1][:-1]

    # parse items within p tag
    columns = row.find_all('p')
    if columns[0].text.strip()[25:] != '':
        new_row['bill_status'] = columns[0].text.strip()[25:]

    all_rows.append(new_row)

#     store info in mongo
#     collection_name.insert_one(new_row)


In [86]:
all_rows[153]

{'leg_id': 'H.R.7058',
 'leg_type': None,
 'leg_url': 'https://www.congress.gov/bill/115th-congress/house-bill/7058?r=154',
 'intro_date': '10/12/2018',
 'congress_id': '115th',
 'desc': 'Digital Goods and Services Tax Fairness Act of 2018',
 'sponsor': 'Rep. Smith, Lamar [R-TX-21]',
 'sponsor_party': None,
 'sponsor_state': None,
 'sponsor_district': None,
 'num_of_cosponsors': '1',
 'cosponsors_url': None,
 'cosponsors': None,
 'committee': 'House - Judiciary',
 'bill_status': 'Introduced',
 'body': None}